### Setup

In [1]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from src.idspy.core.pipeline import PipelineEvent, ObservablePipeline   #pipeline che pubblica eventi su un EventBus
from src.idspy.core.state import State
from src.idspy.core.step import Step
from src.idspy.events.bus import EventBus #sistema di messaggistica per la pubblicazione e sottoscrizione di eventi
from src.idspy.events.events import Event, only_id #decoratore che filtra gli eventi per id

### Create an `EventBus` and register subscribers

In [ ]:
bus = EventBus()
#spiegando i seguenti metodi: 
# log_all: si iscrive a tutti gli eventi e stampa una riga di log con il tipo e l'id dell'evento.
# on_before: si iscrive all'evento BEFORE_STEP e stampa l'indice dello step, i requisiti e cosa fornisce.
# on_before_sum: si iscrive all'evento BEFORE_STEP ma solo per lo step con id "Demo.Sum" e stampa un messaggio specifico.
# on_after: si iscrive all'evento AFTER_STEP e stampa l'indice dello step.
# on_err: si iscrive all'evento ON_ERROR e stampa un messaggio di errore con l'id dello step e il messaggio di errore.

# All events: minimal one-line log
@bus.on(EventBus.ALL)
def log_all(ev: Event) -> None:
    print(f"[ALL] {ev.type} :: {ev.id}")


# Before/After step: show index + requires/provides
@bus.on(PipelineEvent.BEFORE_STEP.value)
def on_before(ev: Event) -> None:
    idx = ev.payload.get("index")
    req = ev.payload.get("requires", [])
    prov = ev.payload.get("provides", [])
    print(f"[BEFORE] idx={idx} step={ev.id} requires={req} provides={prov}")


@bus.on(PipelineEvent.BEFORE_STEP.value, only_id("Demo.Sum"))
def on_before_sum(ev: Event) -> None:
    print(f"[BEFORE] summing step={ev.id}")


#AGGIUNTO
@bus.on(PipelineEvent.AFTER_STEP.value, only_id("Demo.Multiply"))
def on_after_mul(ev: Event) -> None:
        idx = ev.payload.get("index")
        print(f"[AFTER] multipling idx={idx} step={ev.id}")


@bus.on(PipelineEvent.AFTER_STEP.value)
def on_after(ev: Event) -> None:
    idx = ev.payload.get("index")
    print(f"[AFTER]  idx={idx} step={ev.id}")


# Error handler
@bus.on(PipelineEvent.ON_ERROR.value)
def on_err(ev: Event) -> None:
    print(f"[ERROR] step={ev.id} :: {ev.payload.get('error')}")

### Define a couple of simple `Steps`

In [4]:
class Load(Step):
    def __init__(self):
        super().__init__(provides=["data"])

    def run(self, state: State) -> None:
        state["data"] = [1, 2, 3]


class Sum(Step):
    def __init__(self):
        super().__init__(requires=["data"], provides=["sum"])

    def run(self, state: State) -> None:
        state["sum"] = sum(state["data"])


class Boom(Step):
    def __init__(self):
        super().__init__(requires=["missing"])

    def run(self, state: State) -> None:
        # never reached because requires isn't satisfied
        pass

#NUOVE AGGIUNTE
class Multiply(Step):
    def __init__(self, factor: int):
        super().__init__(requires=["data"], provides=["mul"])
        self.factor = factor

    def run(self, state: State) -> None:
        state["mul"] = [x * self.factor for x in state["data"]]

class Pitagora(Step):
    def __init__(self):
        super().__init__(requires=["data"], provides=["pitagora"])

    def run(self, state: State):
        
        if len(state["data"]) < 2:
            state["pitagora"] = None
        else:
            list_pitagora = []
            for i in range(len(state["data"]) - 1):
                a = state["data"][i]
                b = state["data"][i + 1]
                list_pitagora.append(int((a**2 + b**2) ** 0.5))
            state["pitagora"] = list_pitagora     
#FINE NUOVE AGGIUNTE


### Build and run an `ObservablePipeline`

In [5]:
""" p = ObservablePipeline([Load(), Sum()], name="Demo", bus=bus)

s = State()
p(s)
print("STATE:", s.to_dict()) """
# Expected:
# [ALL] PipelineEvent.PIPELINE_START :: Demo
# [BEFORE] idx=0 step=Demo.Load requires=[] provides=['data']
# [ALL] PipelineEvent.BEFORE_STEP :: Demo.Load
# [AFTER]  idx=0 step=Demo.Load
# [ALL] PipelineEvent.AFTER_STEP :: Demo.Load
# [BEFORE] idx=1 step=Demo.Sum requires=['data'] provides=['sum']
# [BEFORE] summing step=Demo.Sum
# [ALL] PipelineEvent.BEFORE_STEP :: Demo.Sum
# [AFTER]  idx=1 step=Demo.Sum
# [ALL] PipelineEvent.AFTER_STEP :: Demo.Sum
# [ALL] PipelineEvent.PIPELINE_END :: Demo
# STATE: {'data': [1, 2, 3], 'sum': 6}

#NUOVE AGGIUNTE
p1 = ObservablePipeline([Load(), Sum(), Multiply(factor=2), Pitagora()], name="Demo", bus=bus)

s1 = State()
p1(s1)
print("STATE:", s1.to_dict())
#fine nuove aggiunte

[ALL] PipelineEvent.PIPELINE_START :: Demo
[BEFORE] idx=0 step=Demo.Load requires=[] provides=['data']
[ALL] PipelineEvent.BEFORE_STEP :: Demo.Load
[AFTER]  idx=0 step=Demo.Load
[ALL] PipelineEvent.AFTER_STEP :: Demo.Load
[BEFORE] idx=1 step=Demo.Sum requires=['data'] provides=['sum']
[BEFORE] summing step=Demo.Sum
[ALL] PipelineEvent.BEFORE_STEP :: Demo.Sum
[AFTER]  idx=1 step=Demo.Sum
[ALL] PipelineEvent.AFTER_STEP :: Demo.Sum
[BEFORE] idx=2 step=Demo.Multiply requires=['data'] provides=['mul']
[ALL] PipelineEvent.BEFORE_STEP :: Demo.Multiply
[AFTER]  idx=2 step=Demo.Multiply
[ALL] PipelineEvent.AFTER_STEP :: Demo.Multiply
[BEFORE] idx=3 step=Demo.Pitagora requires=['data'] provides=['pitagora']
[ALL] PipelineEvent.BEFORE_STEP :: Demo.Pitagora
[AFTER]  idx=3 step=Demo.Pitagora
[ALL] PipelineEvent.AFTER_STEP :: Demo.Pitagora
[ALL] PipelineEvent.PIPELINE_END :: Demo
STATE: {'data': [1, 2, 3], 'sum': 6, 'mul': [2, 4, 6], 'pitagora': [2, 3]}


### Error path (triggers on_error)

In [6]:
p_err = ObservablePipeline([Boom()], name="ErrDemo", bus=bus)

try:
    p_err(State())
except KeyError:
    pass
# Expected:
# [ALL] PipelineEvent.PIPELINE_START :: ErrDemo
# [BEFORE] idx=0 step=ErrDemo.Boom requires=['missing'] provides=[]
# [ALL] PipelineEvent.BEFORE_STEP :: ErrDemo.Boom
# [ERROR] step=ErrDemo.Boom :: KeyError("Boom: missing ['missing']")
# [ALL] PipelineEvent.ON_ERROR :: ErrDemo.Boom
# [ALL] PipelineEvent.PIPELINE_END :: ErrDemo

[ALL] PipelineEvent.PIPELINE_START :: ErrDemo
[BEFORE] idx=0 step=ErrDemo.Boom requires=['missing'] provides=[]
[ALL] PipelineEvent.BEFORE_STEP :: ErrDemo.Boom
[ERROR] step=ErrDemo.Boom :: KeyError("Boom: missing ['missing']")
[ALL] PipelineEvent.ON_ERROR :: ErrDemo.Boom
[ALL] PipelineEvent.PIPELINE_END :: ErrDemo
